![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)



# Collecting Data from Plants with Phidgets

This is a utility notebook to collect data from ["Phidgets"](https://www.phidgets.com/education/learn/projects/plant-kit/) sensors around a plant and store in an online spreadsheet (Ethercalc).

We have tried to make this notebook as friendly as possible for beginner programmers. However, it does require some attention to detail, so follow the instructions carefully. 

The notebook runs both Python code and Javascript (JS) code. It would be nice to do everything in Python, but it seems Javascript is necessary to communicate with the Phidgets in a Jupyter notebook. Fortunately, the Javascript code here is easy enough to read and you will not need to change it.

<h2 style="color: red">IMPORTANT</h2>
You MUST click the <span style="color:red">BIG, RED DISCONNECT BUTTON</span> at the end of this notebook when you are done. This tells the software to release the Phidget device, so it can be used by other notebooks you might try later.

## Overview

The purpose of this notebook is to collect data from certain hardware sensors called "Phidgets" which are set up to monitor the environment of a plant. The data can be viewed directly and it is also saved in an online spreadsheet for later data analysis.

<img src="images/plant2.jpg" alt="A plant with sensor" width="400"/>
<div align="center">

A basement window with our plant.

There are important requirements you must meet in order to complete this setup:
    
- you need the Phidget VINT device, with three sensors attached
    - the temperature/humidity sensor (device type HUM1001_01)
    - the light sensor (device type LUX1000_0)
    - the moisture sensor (device type HUM1100)
- you will need a computer (Mac, Windows, Raspberry Pi) with a USB connection and internet access
- you will need the Chrome or Chromium Web browser (Safari, Firefox, and Edge will not work)

For a step by step guide on how to assemble your Phidgets plant kit , see the following [link](https://www.phidgets.com/education/learn/projects/plant-kit/assemble/)

There are five main steps in this notebook. 

The five steps are:

1. Setting up the gauges 
2. Setting up an online spreadsheet
3. Connect the plant sensors to your computer
4. Setup code to post and display data
5. Start the timer, to continuously post data

***
## Step 1. Setting up the gauges

Run the cells in this notebook, **one cell at a time**. This will give you the chance to respond to any errors, and to chose a unique name for your online storage. 

**Do not select "Run All."**

We first set up some gauges to display values for temperature, humidity, soil moisture and light levels. We also add a text box to display the data as it is read in, and another to show the results of posting to the spreadsheet.  

We first load in all the libraries we need. Click on `▶Run`

In [ ]:
import json, pytz, os.path, random, re, requests, threading
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from time import sleep
from datetime import datetime
from IPython.display import display, Javascript, IFrame, Markdown 

Now we set up the gauges and text boxes. They won't run yet, but should look nices. 

Click on `▶Run`

In [ ]:
# the four sensor gauges
g_temp = go.Indicator(
    mode = "gauge+number",
    value = 20,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Temperature"},
    gauge = {'axis': {'range': [10, 40]}}
)

g_hum = go.Indicator(
    mode = "gauge+number",
    value = 40,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Humidity"},
    gauge = {'axis': {'range': [0, 100]}}
)

g_moist = go.Indicator(
    mode = "gauge+number",
    value = 0.5,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Moisture"},
    gauge = {'axis': {'range': [0, 1.0]}}
)

g_light = go.Indicator(
    mode = "gauge+number",
    value = 40,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Light Level"},
    gauge = {'axis': {'range': [0, 10000]}}
)

fig = make_subplots(
    rows=2,
    cols=2,
    specs=[[{'type' : 'domain'}, {'type' : 'domain'}],[{'type' : 'domain'}, {'type' : 'domain'}]],
    vertical_spacing = 0.25
)
fig.append_trace(g_temp, row=1, col=1)
fig.append_trace(g_hum, row=1, col=2)
fig.append_trace(g_moist, row=2, col=1)
fig.append_trace(g_light, row=2, col=2)

gauges = go.FigureWidget(fig)

# a text widget to monitor the latest data
data_monitor = widgets.Text(
    description='Latest data:',
    value="No recent data yet",
    layout={'width':'500px'},
    disabled = False
)
# a text widget to monitor results of posting to the cloud
post_monitor = widgets.Text(
    description='Post result:',
    value="No post yet",
    layout={'width':'500px'},
    disabled = False
)

def set_gauges(result):  ## This function will update the gauges
    global gauges
    gauges.data[0]['value'] = result['temp']
    gauges.data[1]['value'] = result['humidity']
    gauges.data[2]['value'] = result['moist']
    gauges.data[3]['value'] = result['light']

display(gauges,data_monitor,post_monitor)

***
## Step 2: Setting up the online spreadsheet.

The next step is to create a spreadsheet online where we will store the data. We use a web resource called **EtherCalc.** The most reliable one we have found is at https://ethercalc.nomagic.uk, which is a free, open source project. You might also investigate ethercalc.net or framacalc.org as alternative sites. 

The **base_url** variable below is where we set the web address for the ethercalc storage. 

The following code cell presents a text suggesting a random name for your spreadsheet. You can always make up your own name. If you have already used this notebook, the code will suggest using the same name as last time. But you can alway choose a new one by typing in the text box.

Note this spreadsheet is publically viewable. You should choose a name that is meaningful for you. 

Click on `▶Run`

In [ ]:
base_url = 'https://ethercalc.nomagic.uk/'  # the online spreadsheet home page
file_name = './ss_name.txt'  # the file where we store the spreadsheet name

def get_ss_name():
    if (os.path.isfile(file_name)):
        with open(file_name,'r') as file:
            result=json.load(file)
        if result['name'] != 'default':
            return result
    json_name = {"name" : "Plant_" + "".join(random.choices('0123456789', k=6))}
    with open(file_name,'w') as file:
        json.dump(json_name,file)
        return json_name
    
def set_ss_name(name):
    with open(file_name,'w') as file:
        json.dump({"name" : name},file)
    
def f(SS_Name):
    if len(SS_Name) == 0:
        return "Enter a name here for your spreadsheet."
    if re.search(r'[^A-Za-z0-9_\-]',SS_Name):
        return "ERROR: Use only letters and numbers, no spaces."
    else:
        set_ss_name(SS_Name)
        return "Spreadsheet name is " + SS_Name

print("Choose a name for your spreadsheet here:")
widgets.interact(f, SS_Name=get_ss_name()['name']);

### 2a. Initializing the spreadsheet

Once you have selected a name above, we can initialize the online spreadsheet to receive data. 

Click on `▶Run`

In [ ]:
post_url = base_url + '_/' + get_ss_name()['name']

command_list = [
    'set A1 text t Date',
    'set B1 text t Time',
    'set C1 text t Temperature',
    'set D1 text t Humidity',
    'set E1 text t Moisture',    
    'set F1 text t Luminance',
    'pane row 2'
]

for command in command_list:
    myJSON = {'command':command}
    r = requests.post(post_url, json= myJSON)
    if r.status_code == 202:
        print(f"Command '{command[:6]}' successful")
    else:
        print(f"Command '{command}' failed")



### 2b. Viewing the spreadsheet

Once your spreadsheet has some data in it, it will look something like this:

<img src="images/SS2.png">
<div align="center">
    
The following code cells will access the spreadsheet you created yourself above.

Run this next cell to see your spreadsheet. Click on `▶Run`

In [ ]:
site_url = base_url + get_ss_name()['name'] + '/view'
IFrame(site_url, 700,500)

Run the following code to see the name of your spreadsheet and view it in another window. Click on `▶Run`

In [ ]:
site_url = base_url + get_ss_name()['name'] + '/view'
Markdown(f"We can open your spreadsheet in a separate browser tab by clicking on this link: \n \
\n \
<a href={site_url} target='_blank'>{site_url}</a>")

***
## Step 3: Connecting the sensor devices

At this point, you should connect the Phidgets hardware to your computer. This includes 4 separate devices:
- the Phidget VINT device, attached to the computer's USB port
- the combined temperature/humidity sensor, attached to the VINT
- the soil moisture sensor, attached to the VINT
- the light sensor, attached to the VINT

When you first attach the VINT device to the USB port, your computer may ask you (pop up) whether you wish to connect to this USB device. Please answer **"yes"** to this security request. 

We now go set up the software to communicate with the Phidget devices.

### 3a. Loading libraries

We need to import libraries for Javascript. Click on `▶Run`

In [ ]:
%%js
requirejs.config({
    paths: { 
        'phidget22': ['https://unpkg.com/phidget22/browser/phidget22'], 
    },                                         
});
require(['phidget22'], (phidget22) => {
   window.phidget22 = phidget22; 
});

In [ ]:
## We pause for a second here, to allow some time for the library to load in the background
sleep(1)

### 3b. Opening the USB connection

We now open a connection between the computer and your Phidget VINT device. The VINT must be plugged into your computer's USB port. Run the following cell, and follow the prompts to select the VINT device (a list appears that you should click on). This will pair the device with your computer. 

Click on `▶Run`

In [ ]:
%%js
if (window.usbconn === undefined) {
    element.text("Creating a new USB Connection.");
    window.usbconn = new phidget22.USBConnection();    
    usbconn.connect().then(() => {
        usbconn.requestWebUSBDeviceAccess();
    }).catch(err => {
        window.usbconn.delete();
        element.append("Error connecting to USB" + err);
    });
}

This should open a window that asks you to select the Phidget Hub. Do so, and click "OK." The window looks like this:
<img src="images/Pconnect.png" width=300>
<div align="center">

In [ ]:
## We rest for a bit while the USB connects
sleep(1)

### 3c. Confirm the USB connection 

Run the following code to see if the device is connected. It should say "true." If it does not, check your cable connections. You may also need to check the security settings on your computer to allow new USB devices to get connected. 

Click on `▶Run`

In [ ]:
%%js
element.text("Is the USB device connected? " + usbconn.connected);

### 3d. Connect the sensors

We make a request to open the four different sensors, for temperature, humidity, soil moisture and light level. Be sure your three sensors are plugged into the VINT device. 

If you are missing a sensor or two, that is okay. The data collection for the other sensors will still work. The cells below will connect the sensors, then check to see that they are attached. 

Click on `▶Run`

In [ ]:
%%js

window.tempSensor = new phidget22.TemperatureSensor();
tempSensor.onAttach = function () {
    this.setDataInterval(1000);  
};
tempSensor.onTemperatureChange = function (value) {
    IPython.notebook.kernel.execute(
        "gauges.data[0]['value'] = " + value
    );  
};

window.humSensor = new phidget22.HumiditySensor();
humSensor.onAttach = function () {
    this.setDataInterval(1000);  
};
humSensor.onHumidityChange = function (value) {
    IPython.notebook.kernel.execute(
        "gauges.data[1]['value'] = " + value
    );  
};

window.moistSensor = new phidget22.VoltageRatioInput();
moistSensor.onAttach = function () {
    this.setDataInterval(1000);  
};
moistSensor.onVoltageRatioChange = function (value) {
    IPython.notebook.kernel.execute(
        "gauges.data[2]['value'] = " + value
    );  
};

window.liteSensor = new phidget22.LightSensor();
liteSensor.onAttach = function () {
    this.setDataInterval(1000);  
};
liteSensor.onIlluminanceChange = function (value) {
    IPython.notebook.kernel.execute(
        "gauges.data[3]['value'] = " + value
    );  
};
    
async function setup_sensors() {
    let errorCode = 0;
    try {await tempSensor.open(1000);} catch {errorCode |= 1;}    
    try {await humSensor.open(1000);} catch {errorCode |= 2;}
    try {await moistSensor.open(1000);} catch {errorCode |= 4;}
    try {await liteSensor.open(1000);} catch {errorCode |= 8;} 
    return errorCode
}

setup_sensors()

In [ ]:
## pause for a bit while the sensor get connected
sleep(5)

### 3e. Confirm the sensor connections

Run the following code to see if the four devicess are connected. All four lines should say "YES." If not, check your cable connections. 

Click on `▶Run`

In [ ]:
%%js
element.text("Is the humidity sensor attached? " + (humSensor.attached ? "YES.":"NO."));
element.append("<br>Is the temperature sensor attached? " + (tempSensor.attached ? "YES.":"NO."));
element.append("<br>Is the moisture sensor attached? " + (moistSensor.attached ? "YES.":"NO."));
element.append("<br>Is the light sensor attached? " + (liteSensor.attached ? "YES.":"NO."))

### 3f. Reading the sensors directly

As a check, you can read the values directly with the following code. Let's look at temperature.  Click on `▶Run`

In [ ]:
%%js
try {
    element.text("The temperature is " + tempSensor.temperature);
}
catch {
    element.text("The device is not giving a value.");    
}

## Step 4: Setup code to display and post the data

The following code grabs data from the gauges and posts it onto the online spreadsheet. In the text boxes, it will also display what data was collected and the result of the online post. Results codes like 200 or 202 are good. 

We also set up an independent thread to request a post every 15 minutes. 

Click on `▶Run`

In [ ]:
# set the time zone
tz = pytz.timezone('Canada/Pacific')

# here we grab the data results from gauges (g) then post. 
def post_stuff(g):
    data = datetime.now(tz).strftime("%Y-%m-%d,%H:%M:%S")
    data += ', ' + f'{g.data[0]["value"]:.2f}'
    data += ', ' + f'{g.data[1]["value"]:.2f}'
    data += ', ' + f'{g.data[2]["value"]:.2f}'
    data += ', ' + f'{g.data[3]["value"]:.2f}'
    data_monitor.value = data
    post_url = base_url + '_/' + get_ss_name()['name']
    r = requests.post(post_url, data= data) ## we could return a status code here...
    post_monitor.value = datetime.now(tz).strftime("%Y-%m-%d,%H:%M:%S") + ", Status: " + str(r.status_code)
        
def post_loop(g):
    total = 100*24*4 ## post for 100 days (4 per hour gives 100*24*4 totla)
    for i in range(total):
        post_stuff(g)
        sleep(15*60)  ## Sleep for 15 minutes before next post


thread = threading.Thread(target=post_loop,args=(gauges,))  

***
## Step 5. Start the timer thread

We now start the thread, and it will immediately begin posting data every 15 minutes. 

Click on `▶Run`

In [ ]:
thread.start() # start the loop

You can see the results in the next cell. Click on `▶Run`

In [ ]:
display(gauges,data_monitor,post_monitor)

## YAY. We are all done

The data should be collecting now. You can watch the gauges above, which will update every second or so. 

You can also see the results in the text boxes, and check the online spreadsheet. You can see live in the next cell. It will update every 15 minutes. 

Click on `▶Run`

In [ ]:
site_url = base_url + get_ss_name()['name'] + '/view'
IFrame(site_url, 700,500)

***
## Finishing up. Closing down the sensors

<h2 style="color: red">REALLY IMPORTANT</h2>

Close the sensor now, as otherwise they will keep busy forever, always trying to update the spreadsheet with the latest values. Also, if you just quit the notebook, the sensors may not disconnect properly, which will give you trouble the next time you try to connect.

So, don't skip this next step.

The following cell creates a button that you can click to close the Phidgets. Click it once you are all done with the Phidgets in this notebook. 

Click on `▶Run`

In [ ]:
def doDisconnect(b):
    display(Javascript("""
        (async () => {
            await humSensor.close();
            await tempSensor.close();
            await moistSensor.close();
            await liteSensor.close();
            usbconn.close();
            usbconn.delete();
            delete window.usbconn;
            element.text("You have disconnected the Phidgets.");
        })();
    """))

run_button = widgets.Button(
    description = 'IMPORTANT: Click to disconnect', 
        button_style='danger',layout=widgets.Layout(width='50%', height='80px')
)
print("Press this button when you are done, to disconnect the Phidgets")
run_button.on_click(doDisconnect)

display(run_button)


### Confirm

You can confirm the Phidgets are open or closed by running the following cell. 

If any device is still attached, try clicking the button above, again.

Click on `▶Run`

In [ ]:
%%js
element.text("Is the humidity sensor attached? " + (humSensor.attached ? "YES.":"NO."));
element.append("<br>Is the temperature sensor attached? " + (tempSensor.attached ? "YES.":"NO."));
element.append("<br>Is the moisture sensor attached? " + (moistSensor.attached ? "YES.":"NO."));
element.append("<br>Is the light sensor attached? " + (liteSensor.attached ? "YES.":"NO."))

## Conclusion

We have shown how to run a utility to grab Phidget sensor data and post it on the web. It uses a combination of Javascript and Python to access both the front end and back end engines in the Jupyter notebook environment. 

If you wish to collect data for several days or more, you will need the Jupyter "Hub" software installed on your local computer. If you run this software on the Callysto hub (https://hub.callysto.ca), it will disconnect after a day or so of data collection. 

For additonal support with this notebook, contact M. Lamoureux or Mary Grant (Callysto ambassadors) for details.

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)